# Do some Analysis

In [74]:
import numpy as np
from json import loads, dumps
from os import path, makedirs
import openpyxl

import xml.etree.ElementTree as ElementTree

In [82]:
data = None
with open('results-yash-testing.json', 'r') as dataInputFile:
    data = loads(dataInputFile.read())

excel_wb = openpyxl.Workbook()

In [83]:
# excel lines: [Profile #, filename, aggregateVMAF, aggregatePSNR, aggreggateSSIM, aggreggateMS_SSIM]

# l0: each key is a filename
for filename in data:
    results_for_file = data[filename]
    
    # L1: each key is a codec
    for codec in results_for_file:
        results_for_codec = results_for_file[codec]
        sheetname = codec.replace('/', '-')
        if sheetname not in excel_wb.sheetnames:
            excel_wb.create_sheet(sheetname)
            excel_wb[sheetname].cell(row=1, column=1).value = 'profile #'
            excel_wb[sheetname].cell(row=1, column=2).value = 'filename'
            excel_wb[sheetname].cell(row=1, column=3).value = 'profile'
            excel_wb[sheetname].cell(row=1, column=4).value = 'VMAF'
            excel_wb[sheetname].cell(row=1, column=5).value = 'PSNR'
            excel_wb[sheetname].cell(row=1, column=6).value = 'SSIM'
            excel_wb[sheetname].cell(row=1, column=7).value = 'MS_SSIM'
        
        cur_sheet = excel_wb[sheetname]
        
        i = 0
        for profile in results_for_codec:
            (profile, results) = (profile['profile'], profile['output'])

            p = path.join('.', 'outputs', filename.replace('/', '-'), codec.replace('/', '-'))
            try:
                makedirs(p)
            except:
                pass
            with open(path.join(p, 'profile-' + str(i) + '.xml'), 'w') as fil:
                fil.write(results)
            with open(path.join(p, 'profile-' + str(i) + '-settings.json'), 'w') as fil:
                fil.write(dumps(profile))
            
            xmltree = ElementTree.parse(path.join(p, 'profile-' + str(i) + '.xml'))
            root = xmltree.getroot()
            
            res_vmaf = [a.attrib for a in root.findall('./fyi')][0]
            
            excel_wb[sheetname].append((
                i,
                filename,
                str(profile),
                float(res_vmaf['aggregateVMAF']),
                float(res_vmaf['aggregatePSNR']),
                float(res_vmaf['aggregateSSIM']) * 100.,
                float(res_vmaf['aggregateMS_SSIM']) * 100.
            ))
#            excel_wb[sheetname].append(('prof#', 'fil', 'prof', 'vmaf', 'psnr', 'ssim', 'msssim'))
            
            i += 1

In [84]:
excel_wb.save('results.xlsx')